# **Análisis de Riesgo por Segmentación de Factores Críticos 🏥**

En este cuarto notebook, damos un paso definitivo desde la estadística descriptiva hacia la **inferencia clínica aplicada**. Tras haber explorado la distribución de las variables en los notebooks anteriores, aquí nos enfocamos en responder la pregunta clave: **¿Cuál es el riesgo real de infarto según el rango en el que se encuentra un paciente?**

### **Objetivos de este Análisis:**
1.  **Segmentación por Quintiles:** Dividir las variables continuas (`edad`, `glucosa`, `IMC`) en 5 niveles exactos, desde el valor mínimo real hasta el máximo registrado (0-82 años en edad, por ejemplo).
2.  **Cálculo de Incidencia Real:** Determinar el porcentaje directo de pacientes que sufrieron un infarto en cada segmento, eliminando el sesgo de "suavizado" de otros modelos.
3.  **Validación con Intervalos de Confianza (95%):** Aplicar rigor estadístico mediante barras de error para distinguir entre una tendencia real y una casualidad por falta de datos.
4.  **Identificación de Umbrales de Alerta:** Localizar los "puntos de quiebre" donde el riesgo deja de ser bajo y se convierte en una emergencia médica.

Este análisis transforma datos abstractos en una **herramienta de triaje** capaz de identificar perfiles de alto riesgo antes de entrenar cualquier modelo de IA.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

# 1. Carga del dataset
df = pd.read_csv('dataset/healthcare-dataset-stroke-transformed.csv')

def analisis_5_niveles_exactos(df, col, nombre_visible, color_barras):
    # --- PASO 1: Definir límites basados en el Mínimo y Máximo real ---
    min_val = df[col].min()
    max_val = df[col].max()
    
    # Creamos 6 puntos para generar 5 espacios (bins)
    cortes = np.linspace(min_val, max_val, 6)
    
    # Generamos etiquetas descriptivas
    etiquetas = [f"{int(cortes[i])}-{int(cortes[i+1])}" for i in range(len(cortes)-1)]
    
    df_temp = df.copy()
    # include_lowest=True asegura que el valor mínimo (ej. 0 años) no quede fuera
    df_temp['rango'] = pd.cut(df_temp[col], bins=cortes, labels=etiquetas, include_lowest=True)
    
    # --- PASO 2: Estadísticas de Riesgo ---
    resumen = df_temp.groupby('rango', observed=True)['stroke'].agg(['mean', 'count']).reset_index()
    resumen['incidencia_%'] = (resumen['mean'] * 100).round(1)
    
    # Error estándar (Margen de confianza del 95%)
    resumen['error_±%'] = (1.96 * np.sqrt((resumen['mean'] * (1 - resumen['mean'])) / resumen['count']) * 100).round(1)
    
    # --- PASO 3: Gráfico de Barras ---
    fig = px.bar(
        resumen, x='rango', y='incidencia_%',
        error_y='error_±%',
        title=f'Perfil de Riesgo por {nombre_visible}: 5 Niveles Críticos',
        labels={'incidencia_%': '% Prob. Infarto', 'rango': f'Intervalos de {nombre_visible}'},
        text_auto=True,
        color_discrete_sequence=[color_barras]
    )
    
    fig.update_layout(template='plotly_white', height=450, yaxis_title="Probabilidad de Infarto (%)")
    fig.show()
    
    resumen.insert(0, 'Variable', nombre_visible)
    return resumen[['Variable', 'rango', 'count', 'incidencia_%', 'error_±%']]

# --- EJECUCIÓN ---
t_age = analisis_5_niveles_exactos(df, 'age', 'Edad', '#e74c3c')
t_gluc = analisis_5_niveles_exactos(df, 'avg_glucose_level', 'Glucosa', '#3498db')
t_bmi = analisis_5_niveles_exactos(df, 'bmi', 'IMC', '#27ae60')

# Unificación en Tabla Maestra
reporte_final = pd.concat([t_age, t_gluc, t_bmi]).reset_index(drop=True)
print("\n📋 REPORTE DE INCIDENCIA POR QUINTILES DE VALOR REAL")
display(reporte_final)


📋 REPORTE DE INCIDENCIA POR QUINTILES DE VALOR REAL


,Variable,rango,count,incidencia_%,error_±%
0,Edad,0-16,796,0.3,0.3
1,Edad,16-32,923,0.1,0.2
2,Edad,32-49,1180,1.4,0.7
3,Edad,49-65,1245,5.9,1.3
4,Edad,65-82,965,16.2,2.3
5,Glucosa,55-98,3040,3.6,0.7
6,Glucosa,98-141,1266,3.7,1.0
7,Glucosa,141-185,244,6.6,3.1
8,Glucosa,185-228,426,12.9,3.2
9,Glucosa,228-271,133,15.8,6.2


# **Conclusiones del Análisis de Riesgo 📋**

Tras procesar los 5 niveles de exposición para cada variable clave, los datos arrojan evidencia concluyente para el desarrollo del modelo predictivo:

### **1. La Edad como Predictores Maestro**
* **Resultados:** El riesgo es prácticamente inexistente (0.3% - 0.1%) en los primeros dos quintiles (hasta los 32 años). Sin embargo, en el último quintil (**65-82 años**), la incidencia se dispara al **16.2%**.
* **Validación:** El bajo margen de error (±2.3%) confirma que la edad avanzada es el factor de riesgo más estable y potente del dataset.

### **2. Glucosa: El Efecto "Escalón"**
* **Resultados:** Los niveles de glucosa no muestran un aumento lineal. El riesgo se mantiene estable hasta el cuarto quintil, pero al superar los **185 mg/dL** (Nivel 5), la probabilidad de infarto salta por encima del **13%**.
* **Implicación:** El modelo de IA debe tratar la hiperglucemia como un activador crítico de riesgo, no solo como una variable continua más.

### **3. IMC: La Paradoja de la Obesidad**
* **Resultados:** A diferencia de la edad y la glucosa, el IMC muestra una correlación más débil. El riesgo aumenta ligeramente en niveles de sobrepeso (4% a 6.6%), pero curiosamente disminuye en los rangos de obesidad extrema registrados.
* **Interpretación:** Esto sugiere que, en esta población, el peso es un factor secundario que probablemente requiere de la presencia de hipertensión o edad avanzada para manifestarse como un infarto.


### **Veredicto Final para el Modelado:**
Este notebook confirma que los mejores predictores para nuestro modelo de Machine Learnig serán la **Edad** y la **Glucosa**, especialmente cuando se encuentran en sus quintiles superiores. 